In [127]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import requests
import time
import datetime

from pandas import Series, DataFrame

import os
from random import randrange

# Date And Time
now = datetime.datetime.now()
nowDateTime = now.strftime('%Y%m%d%H%M%S')

path = 'C:/kdeWorking/Project/DL/DL/crawling/chromedriver'
driver = webdriver.Chrome(path)

url = 'https://finance.daum.net/domestic'
driver.get(url)
resp = requests.get(url)
time.sleep(randrange(10,40)/10)

In [24]:
TOP_UP_LIST = []
cnt = 0

In [3]:
# isDir and mkDir
def ismkDir(name):
    dirPath = '.\\{}'.format(name)
    if os.path.exists(dirPath) is False:
        os.mkdir(dirPath)

In [126]:
driver.close()

In [125]:
TOP_UP_LIST = []
cnt = 0

In [121]:
# function Make_csv
def make_csv(TOP_UP_LIST, PATH):
    print('make_csv in')
#     # Date And Time
#     now = datetime.datetime.now()
#     nowDateTime = now.strftime('%Y%m%d%H%M%S')
    global nowDateTime
    
    raw_data = {
        'UP_DOWN_TOP_KOSPI_TITLE':    TOP_UP_LIST[0],
        'UP_DOWN_TOP_KOSPI_PER':      TOP_UP_LIST[1],
        'UP_DOWN_TOP_KOSDAQ_TITLE':   TOP_UP_LIST[2],
        'UP_DOWN_TOP_KOSDAQ_PER':     TOP_UP_LIST[3],
        'RISING_STOCK_KOSPI_TITLE':   TOP_UP_LIST[4] ,
        'RISING_STOCK_KOSPI_PER':     TOP_UP_LIST[5] ,
        'RISING_STOCK_KOSDAQ_TITLE':  TOP_UP_LIST[6] ,
        'RISING_STOCK_KOSDAQ_PER':    TOP_UP_LIST[7] ,
        'FOREIGNER_BUY_KOSPI_TITLE':  TOP_UP_LIST[8] ,
        'FOREIGNER_BUY_KOSPI_PER':    TOP_UP_LIST[9] ,
        'FOREIGNER_BUY_KOSDAQ_TITLE': TOP_UP_LIST[10] ,
        'FOREIGNER_BUY_KOSDAQ_PER':   TOP_UP_LIST[11] ,
        'ORGAN_BUY_KOSPI_TITLE':      TOP_UP_LIST[12] ,
        'ORGAN_BUY_KOSPI_PER':        TOP_UP_LIST[13] ,
        'ORGAN_BUY_KOSDAQ_TITLE':     TOP_UP_LIST[14] ,
        'ORGAN_BUY_KOSDAQ_PER':       TOP_UP_LIST[15]
    }
    print('make_csv raw_data after')
    data = DataFrame(raw_data)
    data.to_csv('.\\'+PATH+'\\'+nowDateTime+'.csv')
    
    TOP_UP_LIST = []
    cnt = 0
    
    print('make_csv_TOP_UP finish')

In [120]:
def make_csv_KOS(arr, PATH):
    
    global nowDateTime
    
    time.sleep(randrange(10,40)/10)
    
    raw_data = {
        'KOSPI_NUM':          [arr[0]],
        'KOSPI_UP_DOWN_NUM':  [arr[1]],
        'KOSPI_UP_DOWN_PER':  [arr[2]],
        'KOSDAQ_NUM':         [arr[3]],
        'KOSDAQ_UP_DOWN_NUM': [arr[4]],
        'KOSDAQ_UP_DOWN_PER': [arr[5]]
    }
    time.sleep(randrange(10,40)/10)
    data = DataFrame(raw_data)
    data.to_csv('.\\'+PATH+'\\'+nowDateTime+'.csv')
    
    print('make_csv_KOS finish')

In [119]:
def KOS_LOGIC():
    
    arr = []
    
    # 코스피 지수
    KOSPI_SELECTOR = driver.find_elements_by_css_selector('.on')
    KOSPI_LIST = KOSPI_SELECTOR[1].text.split("\n")
    # 코스닥 지수
    KOSDAQ_SELECTOR = driver.find_element_by_css_selector('.off')
    KOSDAQ_LIST = KOSDAQ_SELECTOR.text.split('\n')
    
    time.sleep(randrange(10,40)/10)
    
    # KOSPI_NUM
    arr.append(KOSPI_LIST[1])
    # KOSPI_UP_DOWN_NUM
    arr.append(KOSPI_LIST[2])
    # KOSPI_UP_DOWN_PER
    arr.append(KOSPI_LIST[3])

    # KOSDAQ_NUM
    arr.append(KOSDAQ_LIST[1])
    # UP_DOWN_NUM
    arr.append(KOSDAQ_LIST[2])
    # UP_DOWN_PER
    arr.append(KOSDAQ_LIST[3])
    
    time.sleep(randrange(10,40)/10)
    
    make_csv_KOS(arr, 'KOS')

In [37]:
def TOP_LOGIC():
    
    # 등락률 상위업종
    # KOSPI_TITLE, PER / KOSDAQ_TITLE, PER
    UP_DOWN_TOP_TABLE = driver.find_element_by_tag_name('table')

    time.sleep(randrange(10,40)/10)

    TOP_UP(UP_DOWN_TOP_TABLE)

    # 상승종목
    # KOSPI_TITLE, PER / KOSDAQ_TITLE, PER
    RISING_STOCK_TABLE = driver.find_elements_by_tag_name('table')[1]

    time.sleep(randrange(10,40)/10)

    TOP_UP(RISING_STOCK_TABLE)

    # 외국인 순매수
    # KOSPI_TITLE, PER / KOSDAQ_TITLE, PER
    FOREIGNER_BUY_TABLE = driver.find_elements_by_tag_name('table')[2]

    time.sleep(randrange(10,40)/10)

    TOP_UP(FOREIGNER_BUY_TABLE)


    # 기관 순매수
    # KOSPI_TITLE, PER / KOSDAQ_TITLE, PER
    ORGAN_BUY_TABLE = driver.find_elements_by_tag_name('table')[3]

    time.sleep(randrange(10,40)/10)

    TOP_UP(ORGAN_BUY_TABLE)


In [5]:
# function 등락률 상위업종, 상승종목, 외국인 순매수, 기관 순매수
TOP_UP_LIST = []
cnt = 0
def TOP_UP(TABLE):
    global TOP_UP_LIST
    global cnt
    
    _TR = TABLE.find_elements_by_tag_name('tr')
    
    KOSPI_TITLE = []
    KOSPI_PER = []
    KOSDAQ_TITLE = []
    KOSDAQ_PER = []
    
    temp_title = []
    temp_per = []
    
    # tr[0] : 코스피 코스닥 삭제
    del _TR[0]
    
    # title, per로 구분
    for i in _TR:
        
        tr_list = i.text.split('\n')

        for j in range(0,4):
            if j % 2 == 0:
                temp_title.append(tr_list[j])
            else:
                temp_per.append(tr_list[j])
    
    # KOSPI_title, per / KOSDAQ_title, per 로 구분
    for i in range(0,10):
        if i % 2 == 0:
            KOSPI_TITLE.append(temp_title[i])
            KOSPI_PER.append(temp_per[i])
        else:
            KOSDAQ_TITLE.append(temp_title[i])
            KOSDAQ_PER.append(temp_per[i])
    
    TOP_UP_LIST.append(KOSPI_TITLE)
    TOP_UP_LIST.append(KOSPI_PER)
    TOP_UP_LIST.append(KOSDAQ_TITLE)
    TOP_UP_LIST.append(KOSDAQ_PER)
    
    cnt += 1
    
    if cnt == 4:
        make_csv(TOP_UP_LIST,'TOP_UP')
    
#     return KOSPI_TITLE, KOSPI_PER, KOSDAQ_TITLE, KOSDAQ_PER

In [175]:
# function 등락률 하위업종
def UP_DOWN_BOTTOM(UP_DOWN_BOTTOM_TR):
    
    UP_DOWN_BOTTOM_KOSPI_TITLE = []
    UP_DOWN_BOTTOM_KOSPI_PER = []
    UP_DOWN_BOTTOM_KOSDAQ_TITLE = []
    UP_DOWN_BOTTOM_KOSDAQ_PER = []
    
    temp_title = []
    temp_per = []
    
    # tr[0] : 코스피 코스닥 삭제
    del UP_DOWN_BOTTOM_TR[0]
    
    # title, per로 구분
    for i in UP_DOWN_BOTTOM_TR:
        
        tr_list = i.text.split('\n')

        for j in range(0,4):
            if j % 2 == 0:
                temp_title.append(tr_list[j])
            else:
                temp_per.append(tr_list[j])
    
    # KOSPI_title, per / KOSDAQ_title, per 로 구분
    for i in range(0,10):
        if i % 2 == 0:
            UP_DOWN_BOTTOM_KOSPI_TITLE.append(temp_title[i])
            UP_DOWN_BOTTOM_KOSPI_PER.append(temp_per[i])
        else:
            UP_DOWN_BOTTOM_KOSDAQ_TITLE.append(temp_title[i])
            UP_DOWN_BOTTOM_KOSDAQ_PER.append(temp_per[i])
    
    return UP_DOWN_BOTTOM_KOSPI_TITLE, UP_DOWN_BOTTOM_KOSPI_PER, UP_DOWN_BOTTOM_KOSDAQ_TITLE, UP_DOWN_BOTTOM_KOSDAQ_PER

# main

In [128]:
ismkDir('KOS')
ismkDir('TOP_UP')
ismkDir('BOTTOM_DOWN')

# 하락 종목
DECLINE_STOCK_TITLE = []
DECLINE_STOCK_PER = []

# 외국인 순매도
FOREIGNER_SELL_TITLE = []
FOREIGNER_SELL_PER = []

# 기관 순매수
ORGAN_BUY_TITLE = []
ORGAN_BUY_PER = []

# 기관 순매도
ORGAN_SELL_TITLE = []
ORGAN_SELL_PER = []

print('kos_logic before')
time.sleep(randrange(10,40)/10)
KOS_LOGIC()
time.sleep(randrange(10,40)/10)
print('kos_logic after')

print('top_logic before')
time.sleep(randrange(10,40)/10)
TOP_LOGIC()
time.sleep(randrange(10,40)/10)
print('top_logic after')

# 등락률 하위업종 클릭
# time.sleep(randrange(10,40)/10)
# ul_tag = driver.find_elements_by_css_selector('.box_tabs')[1]
# ul_a_tag = ul_tag.find_elements_by_tag_name('a')
# ul_a_tag[1].click()

# 등락률 하위업종
# KOSPI_TITLE, PER / KOSDAQ_TITLE, PER
# UP_DOWN_BOTTOM_TABLE = driver.find_element_by_tag_name('table')
# UP_DOWN_BOTTOM_TR = UP_DOWN_BOTTOM_TABLE.find_elements_by_tag_name('tr')

# time.sleep(randrange(10,40)/10)

# UP_DOWN_BOTTOM_KOSPI_TITLE, UP_DOWN_BOTTOM_KOSPI_PER, UP_DOWN_BOTTOM_KOSDAQ_TITLE, UP_DOWN_BOTTOM_KOSDAQ_PER = UP_DOWN_BOTTOM(UP_DOWN_BOTTOM_TR)



# print(RISING_STOCK_KOSPI_TITLE)
# print(RISING_STOCK_KOSPI_PER)
# print(RISING_STOCK_KOSDAQ_TITLE)
# print(RISING_STOCK_KOSDAQ_PER)

kos_logic before
make_csv_KOS finish
kos_logic after
top_logic before
make_csv in
make_csv raw_data after
make_csv_TOP_UP finish
top_logic after


In [129]:
# 등락률 하위업종 클릭
ul_tag = driver.find_elements_by_css_selector('.box_tabs')[1]
ul_a_tag = ul_tag.find_elements_by_tag_name('a')
ul_a_tag[1].click()

In [130]:
# 하락종목 클릭
ul_tag = driver.find_elements_by_css_selector('.box_tabs')[2]
ul_a_tag = ul_tag.find_elements_by_tag_name('a')
ul_a_tag[1].click()

In [131]:
# 외국인 순매도
ul_tag = driver.find_elements_by_css_selector('.box_tabs')[3]
ul_a_tag = ul_tag.find_elements_by_tag_name('a')
ul_a_tag[1].click()

In [133]:
# 기관 순매도
ul_tag = driver.find_elements_by_css_selector('.box_tabs')[4]
ul_a_tag = ul_tag.find_elements_by_tag_name('a')
ul_a_tag[1].click()

In [ ]:
# function 등락률 하위업종, 하락종목, 외국인 순매도, 기관 순매도
TOP_UP_LIST = []
cnt = 0
def BOTTOM_DOWN(TABLE):
    global TOP_UP_LIST
    global cnt
    
    _TR = TABLE.find_elements_by_tag_name('tr')
    
    KOSPI_TITLE = []
    KOSPI_PER = []
    KOSDAQ_TITLE = []
    KOSDAQ_PER = []
    
    temp_title = []
    temp_per = []
    
    # tr[0] : 코스피 코스닥 삭제
    del _TR[0]
    
    # title, per로 구분
    for i in _TR:
        
        tr_list = i.text.split('\n')

        for j in range(0,4):
            if j % 2 == 0:
                temp_title.append(tr_list[j])
            else:
                temp_per.append(tr_list[j])
    
    # KOSPI_title, per / KOSDAQ_title, per 로 구분
    for i in range(0,10):
        if i % 2 == 0:
            KOSPI_TITLE.append(temp_title[i])
            KOSPI_PER.append(temp_per[i])
        else:
            KOSDAQ_TITLE.append(temp_title[i])
            KOSDAQ_PER.append(temp_per[i])
    
    TOP_UP_LIST.append(KOSPI_TITLE)
    TOP_UP_LIST.append(KOSPI_PER)
    TOP_UP_LIST.append(KOSDAQ_TITLE)
    TOP_UP_LIST.append(KOSDAQ_PER)
    
    cnt += 1
    
    if cnt == 4:
        make_csv(TOP_UP_LIST,'TOP_UP')
    
#     return KOSPI_TITLE, KOSPI_PER, KOSDAQ_TITLE, KOSDAQ_PER

In [106]:
driver.close()